# Analyzing borrowers’ risk of defaulting

This project is to prepare a report for a bank’s loan division to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

The report will be considered when building a credit scoring of a potential customer. A credit scoring is used to evaluate the ability of a potential borrower to repay their loan.

## General Data Information

In [25]:
#Import pandas library and bring in data file to data variable
import pandas as pd
data = pd.read_csv('/datasets/credit_scoring_eng.csv')

#Display general information regarding the data set and a look at a fragment of the entries
data.info() 
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


The general information on the customer data indicated there are 21,525 individual entries. Each entry contains the following attributes as defined by the data frame columns:

- `children`: The number of children in the family
- `days_employed`: how long the customer has been working
- `dob_years`: the customer's age
- `education`: the customer's education level
- `education_id`: identifier for the customer's education
- `family_status`: the customer's marital status
- `family_status_id`: identifier for the customer's marital status
- `gender`: the customer's gender
- `income_type`: the customer's income type
- `debt`: whether the customer has ever defaulted on a loan
- `total_income`: monthly income
- `purpose`: reason for taking out a loan

Two particular attributes in the general information immediately draw attention: the `days_employed` and `total_income`. Both of these attributes appear to be missing 2174 entries.

## Data preprocessing

### Processing missing values

To begin, the missing values that were discovered during the look at the general data information must be addressed.

Since the number of the `days_employed` missing values is equal to the number of `total_income` missing values; it is predicted that the missing values correspond to customers who did not have a job, and thus filled out N/A of some sorts for the monthly income and number of days employed at the time of the query. Based on this prediction, the missing values are initially replaced with a 0 value, as that is a representative value for a customer lacking a job and a monthly income amount. To verify the assumption that the missing values are jobless people, the number of customers containing a '0' value for both the `days_employed` and `total_income` attributes is counted:

In [26]:
#Replace all NaN values with a value of zero
data['total_income'] = data['total_income'].fillna(value = 0)
data['days_employed'] = data['days_employed'].fillna(value = 0)

#Determines the number of customers that are jobless (zero days employed and zero total income)
jobless = data[(data['days_employed'] == 0) & (data['total_income'] == 0)]
print('The number of jobless customers is {}'.format(len(jobless)))

The number of jobless customers is 2174


The number of jobless customers equals 2174, which was the number of missing values initially in these two columns. This seems to confirm the initial prediction. A fragment of the `jobless` DataFrame is printed to further investigate. 

In [27]:
jobless.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,0.0,65,secondary education,1,civil partnership,1,M,retiree,0,0.0,to have a wedding
26,0,0.0,41,secondary education,1,married,0,M,civil servant,0,0.0,education
29,0,0.0,63,secondary education,1,unmarried,4,F,retiree,0,0.0,building a real estate
41,0,0.0,50,secondary education,1,married,0,F,civil servant,0,0.0,second-hand car purchase
55,0,0.0,54,secondary education,1,civil partnership,1,F,retiree,1,0.0,to have a wedding


Looking at the `income_type` for the 'jobless' customers, it is expected that most would be unemployed or retirees, which is not the case. Although there are several retirees, there are also civil servants, business owners, and regular employees listed. There are a few potential reasons this may be the case:
- The initial assumption that these were jobless people was incorrect and there was some other error with gathering the data that resulted in missing values.
- Some of the customers are volunteers that do not have an income.
- Customers who are unemployed refrained from actually stating they are unemployed for fear of immediately being turned down for a loan.

**Conclusion**

Regardless of the reason for the missing values, these attribute values will remain a '0' value for these customers as it seems like an appropriate replacement without knowing the customers true `total_income`. Although these missing values make up a fair amount of the overall number of data entries (~10%), a customer's total income is only one of the attributing factors for determining whether the customer will default on a loan.

An alternative approach to replacing the missing values could have been to replace the missing values with the mean or median of the other customer's data. This may have misrepresented the customer's with average incomes if the customer with missing value `total_income` had a higher likelihood of defaulting on a loan. The exact impact of doing this could be quantified; however, the benefit is not worth the time.

### Data type replacement

Two of the attributes in the data set, `days_employed` and `total_income`, are of the float data type. It is much cleaner to make these values an integer data type, seeing as there is limited benefit in knowing the fraction of days in `days_employed` or the number of cents in a customer's `total_income`. A fragment of the DataFrame is printed to verify the change:

In [28]:
#Convert float values to integers
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,bachelor's degree,0,married,0,F,employee,0,40620,purchase of the house
1,1,-4024,36,secondary education,1,married,0,F,employee,0,17932,car purchase
2,0,-5623,33,Secondary Education,1,married,0,M,employee,0,23341,purchase of the house
3,3,-4124,32,secondary education,1,married,0,M,employee,0,42820,supplementary education
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378,to have a wedding


It is noted that a majority of the `days_employed` attribute values are negative. This is most likely due to how this number was generated from the initial query. It is expected that the customer was requested to provide either the known number of days worked or pick the date he or she began working. For the latter, it is anticipated that the `days_employed` value was calculated by taking the customer's provided start date minus the current date, which results in a negative number.

To correct this, the absolute value of each `days_employed` value is used.

In [29]:
data['days_employed'] = abs(data['days_employed'])

Also noted in the `days_employed` attribute are values that correspond to an impractical amount of working days (well over 100 years of days employed). The DataFrame is sorted to show all entries that correspond to `days_employed` that exceeds 100 years to investigate further.

In [30]:
#Show a fragment of data where the days_employed is greater than 100 years
data[data['days_employed'] > 36500].head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378,to have a wedding
18,0,400281,53,secondary education,1,widow / widower,2,F,retiree,0,9091,buying a second-hand car
24,1,338551,57,secondary education,1,unmarried,4,F,retiree,0,46487,transactions with commercial real estate
25,0,363548,67,secondary education,1,married,0,M,retiree,0,8818,buy real estate
30,1,335581,62,secondary education,1,married,0,F,retiree,0,27432,transactions with commercial real estate


It appears that the extremely high `days_employed` values are all for retired customers. It is assumed that this is a result of how the data was gathered and recorded. This problem is noted, but there are no attempts to correct it at this moment as the `days_employed` attribute is rarely used and there appears to be limited benefit in doing so.

**Conclusion**

The two attribute columns `days_employed` and `total_income` are changed from float data types to integer data types. This is not a crucial change for the data at hand, but it does make the data easier to read and compare. While modifying these attribute columns, there were two other errors that were fixed or noted: the negative value for `days_employed` and the unrealistic `days_employed` values.

### Processing duplicates

Prior to processing the duplicate data entries, all case-sensitive values must be converted to a consistent case. It was observed that during the previous data fragment prints that the `education` attribute had an inconsistent case choice. For completeness, all case-sensitive attributes (`education`, `family_status`, `gender`, `income_type` and `purpose`), are converted to all lowercase. This prevents missing any duplicate entries.

In [31]:
#Change all string attributes to lowercase
case_values = ['education', 'family_status', 'gender', 'income_type', 'purpose']

for attribute in case_values:
    data[attribute] = data[attribute].str.lower()

Now that all data entries are case consistent, the number of duplicated data entries can be determined. Only entries that have a duplicate match for every single attribute are counted.

In [32]:
#Find number of duplicate entries
print('The total number of duplicate entries is {}'.format(data.duplicated().sum()))

The total number of duplicate entries is 71


With 71 duplicate entries, it is safe to assume that nearly all of these are errors and can be safely removed as the chances of this realistically happening is close to zero, given the provided attributes.

In [33]:
#Remove duplicate entries
data = data.drop_duplicates().reset_index(drop = True)

**Conclusion**

Duplicate data could have been present as a result of a system error or that the bank had the same customer data from multiple transactions or queries at different times. An additional check for duplicated data could be completed after the customer `purpose` is categorized in the event that the bank had data on the same customer with two different `purpose` values; however, the chance of this seems unlikely and may accidentally remove real data.

### Categorizing Data

The customer data is best categorized according to each customer's purpose, and to place each customer in a certain income bracket since these two attributes have a potential for a significant number of unique responses.

#### Customer Purpose Categorization

To begin categorizing the customer's purpose, the various purposes are first evaluated.

In [34]:
#Print a list of every single customer purpose provided
data['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

From the unique list of all customer purposes, the purposes can generally be divided into four distinct categories:
- Education
- Car Purchase
- Wedding
- Housing

To analyze the customer data easier, each customer response is filtered into one of these four categories.

In [35]:
#Purpose of this function is to take each customer's purpose from the DataFrame and categorize into into one of four
#categories: education, car purchase, wedding, or housing. The function matches certain keywords in the purpose string
#in order to sort into the four categories. Keywords 'hous' and 'educa' are utilized in order to categorize strings that contain
#both 'housing' or 'house', or 'education' and 'educate'. An 'other' category is provided should any of the customer
#purposes not be categorized into one of the four main categories.

def purpose_group(purpose):
    if 'car' in purpose:
        return 'car purchase'
    if 'wedding' in purpose:
        return 'wedding'
    if 'educa' in purpose:
        return 'education'
    if 'university' in purpose:
        return 'education'
    if 'hous' in purpose:
        return 'housing'
    if 'estate' in purpose:
        return 'housing'
    if 'property' in purpose:
        return 'housing'
    return 'other'


#Apply the purpose group function to all data values in the 'purpose' attribute column
data['purpose'] = data['purpose'].apply(purpose_group)

To verify that the function separated as intended. All unique values for the `purpose` column are returned. Only the four categories should be shown if the function worked correctly.

In [36]:
data['purpose'].unique()

array(['housing', 'car purchase', 'education', 'wedding'], dtype=object)

#### Income Bracket Categorization

To categorize each customer into an income bracket, the various levels of the income bracket must first be established. To better understand the range of `total_income`, the max, mean, and median values of the `total_income` attribute are determined.

In [37]:
#Prints the maximum, average, and median total incomes
print('The maximum customer income is {:.0f}.'.format(data['total_income'].max()))
print('The average customer income is {:.0f}.'.format(data['total_income'].mean()))
print('The median customer income is {:.0f}.'.format(data['total_income'].median()))

The maximum customer income is 362496.
The average customer income is 24161.
The median customer income is 21724.


Now that the `total_income` attribute is better understood, the various levels of the income bracket can be established. Since the range of customer incomes is so large, the average income is selected as the 'midpoint' in the income bracket since it is the larger of the mean and median. Since there were also missing values for multiple customers that were replaced with a 0 dollar income; an unknown bracket will be used for those customers. The income bracket levels are as follows:

- unknown: 0
- low: greater than 0 to 18,120 (75% or less of the average `total_income`)
- middle: 18,121 to 30,201 (75% to 125% of the average `total_income`)
- high: 30,202 to 60,403 (125% to 250% of the average `total_income`)
- wealthy: Greater than 60,403

Each of the customer's `total_income` is filtered into one of these five income bracket levels.

In [38]:
#Purpose of this function is to take each customer's total_income from the DataFrame and categorize into into one of four
#income levels: low, middle, high, and wealthy. The function sets a mean value based on the mean of all the values in the
#total_income attribute. Each customer's total_income is then mathematically compared to the mean value to separate them into
#the respective income bracket level. An 'other' category is provided should any of the customer's total_incomes not meet
#one of the four income bracket levels.

def income_group(total_income):
    mean = data['total_income'].mean()
    if total_income == 0:
        return 'unknown'
    if total_income < 0.75*mean:
        return 'low'
    if total_income >= 2.5*mean:
        return 'wealthy'
    if 0.75*mean <= total_income < 1.25*mean:
        return 'middle'
    if 1.25*mean <= total_income < 2.5*mean:
        return 'high'
    return 'other'

#Apply the income group function to all data values in the 'total_income' attribute column
data['total_income'] = data['total_income'].apply(income_group)

**Conclusion**

Both categorization functions worked as intended, separating each of the attributes with a large number of unique values into a much more manageable number of four unique categories. This will make analyzing the data set significantly easier.

## Data Analysis

### Impact of Children

The relationship between having children and repaying a loan on time is evaluated by creating a pivot table showing the number of customers who have defaulted on a loan for each number of children vs. the number of overall customers for each number of children. In addition, a new column is created in the pivot table to determine the percentage of customers that defaulted on a loan. A percentage is required to accurately quantify the data rather than just taking the pure sum of defaulted loans per number of children.

In [39]:
#Purpose of this function is to check if a customer has children or not. Customer's with children will be identified by a 1 in
# the child_check column
def children_check(number_of_children):
    if number_of_children == 0:
        return 0
    else:
        return 1

data['child_check'] = data['children'].apply(children_check)

#Create a pivot table for the child check index in terms of the default likelihood
data_grouped_kids = data.groupby(['child_check']).agg({'debt':['count','sum']})
data_grouped_kids ['percentage, %'] = data_grouped_kids['debt','sum']/data_grouped_kids['debt','count'] * 100
data_grouped_kids.head()

debt       percentage, %
             count   sum              
child_check                           
0            14091  1063      7.543822
1             7363   678      9.208203

The data in the table above indicated that customers with children (one or more) are almost 2% more likely to default on a loan than customers with zero children.

Although the initial question of whether there is a relationship between having kids and repaying a loan was answered, this data could be even further processed. For example, customers who are retirees must also provide the number of children they have; however, a majority of their children would likely no longer be living with the retiree and would have their own job and source of income. These children would, in most cases, not be financially supported by the customer and would have minimal effect on whether the customer defaults on a loan or not. Additional investigation into the data would be required to fully quantify this; however, the benefit is not worth the time seeing as the number of children is only one piece of the puzzle for determining if a customer will default on a loan. 

### Impact of Marital Status

The relationship between marital status and repaying a loan on time is evaluated by creating a pivot table showing the number of customers who have defaulted on a loan for each form of marital status. Similar to the previous pivot table, a new column is created in the pivot table to determine the percentage of customers that defaulted on a loan to better represent the data.

In [40]:
#Creates pivot table for customer family status
data_grouped_marriage = data.groupby(['family_status']).agg({'debt':['count','sum']})
data_grouped_marriage['percentage, %'] = data_grouped_marriage['debt','sum']/data_grouped_marriage['debt','count'] * 100
data_grouped_marriage.sort_values(['percentage, %'], ascending = False).head()

debt      percentage, %
                   count  sum              
family_status                              
unmarried           2810  274      9.750890
civil partnership   4151  388      9.347145
married            12339  931      7.545182
divorced            1195   85      7.112971
widow / widower      959   63      6.569343

The data in the table above indicates that civil partnerships and unmarried customers are more likely to default on a loan. Widows /widowers customers are least likely to default on a loan, and divorced or married customers fall in the middle. These likelihoods seem to be somewhat dependent on the household income. For example, an unmarried customer has only one source of income to repay a loan whereas a married customer could have two sources of income that can be utilized to repay the loan. The civil partnership likelihood is interesting in that one would think this would be closer to the likelihood for married customers, but it is significantly higher. There is no explanation at the moment for why this is the case.

### Impact of Income Level

The relationship between income level and repaying a loan on time is evaluated by creating a pivot table showing the number of customers who have defaulted on a loan for each income bracket level previously established. Similar to the previous pivot table, a new column is created in the pivot table to determine the percentage of customers that defaulted on a loan to better represent the data.

In [41]:
#Creates pivot table for the five income brackets
data_grouped_income = data.groupby(['total_income']).agg({'debt':['count','sum']})
data_grouped_income['percentage, %'] = data_grouped_income['debt','sum']/data_grouped_income['debt','count'] * 100
data_grouped_income.sort_values(['percentage, %'], ascending = False).head()

debt      percentage, %
             count  sum              
total_income                         
middle        7426  652      8.779962
unknown       2103  170      8.083690
low           6089  489      8.030875
high          5183  393      7.582481
wealthy        653   37      5.666156

The data in the table above indicates that the higher the income bracket that a customer is in, the less likely they are to default on a loan. These results are expected since a large income would usually mean it is easier to repay a loan. However, this is also contingent on the amount of the loan and the responsibility of a customer for taking out a loan that they are capable of realistically paying back.

### Impact of Loan Purpose

The relationship between loan purpose and repaying a loan on time is evaluated by creating a pivot table showing the number of customers who have defaulted on a loan for each categorized purpose previously established. Similar to the previous pivot table, a new column is created in the pivot table to determine the percentage of customers that defaulted on a loan to better represent the data.

In [42]:
#Creates a pivot table for the different loan purposes
data_grouped_purpose = data.groupby(['purpose']).agg({'debt':['count','sum']})
data_grouped_purpose['percentage, %'] = data_grouped_purpose['debt','sum']/data_grouped_purpose['debt','count'] * 100
data_grouped_purpose.sort_values(['percentage, %'], ascending = False).head()

debt      percentage, %
              count  sum              
purpose                               
car purchase   4306  403      9.359034
education      4013  370      9.220035
wedding        2324  186      8.003442
housing       10811  782      7.233373

The data in the table above indicates that customers who take out loans intended for car purchases or education have the highest chance of failing to repay the loan, whereas housing loans have the lowest chance, and loans for weddings fall in the middle. 

Initially it was assumed that housing loans would result in the highest chance of defaulting, simply because the amount of these loans are significantly higher compared to the other categories. However, housing and real estate is known to retain value, so even if a customer fails to meet the payments on the loan, the housing or real estate could be sold without much loss to repay the loan. A car, on the other hand, is notorious for depreciating quickly, which would be detrimental for a customer who is unable to make the loan payments. Likewise, for student loans, unless the recent graduate gets a well-paying job right out of college, the high cost of a university degree is often difficult to pay back. Weddings fall into the middle range likely because they are not quite as expensive as education, but do not hold any value to offset the cost of the loan.

## Overall Conclusion

The purpose of this project was to determine how certain attributes of a customer's credit worthiness profile impact the ability of the customer to pay back a loan.

The bank provided existing data regarding customer's credit worthiness that was utilized to determine the relationships between the various customer attributes and the customer's ability to pay back the loan. This data was preprocessed to replace missing values (either data gathering errors or jobless customers), remove duplicate data entries, and to categorize some of the broader attributes to aid in analyzing the data. 

To determine the correlation between specific customer attributes and the ability to pay back a loan, various pivot tables highlighting the bank's existing customer credit worthiness data were utilized to determine the overall percentage of customers that defaulted on loans with respect to the various attributes considered: children, marital status, income level, and loan purpose. A summary of the results is provided in the tables below:

| Has Children?      | Percentage of customers  that default on load |
|--------------------|-----------------------------------------------|
| Yes                | 9.21%                                         |
| No                 | 7.54%                                         |

| Marital Status    | Percentage of customers  that default on load |
|-------------------|-----------------------------------------------|
| Unmarried         | 9.75%                                         |
| Civil Partnership | 9.35%                                         |
| Married           | 7.55%                                         |
| Divorced          | 7.11%                                         |
| Widow / Widower   | 6.57%                                         |

| Income Bracket | Percentage of customers  that default on load |
|----------------|-----------------------------------------------|
| Middle         | 8.78%                                         |
| Unknown        | 8.08%                                         |
| Low            | 8.03%                                         |
| High           | 7.58%                                         |
| Wealthy        | 5.67%                                         |

| Loan Purpose | Percentage of customers  that default on load |
|--------------|-----------------------------------------------|
| Car Purchase | 9.36%                                         |
| Education    | 9.22%                                         |
| Wedding      | 8.00%                                         |
| Housing      | 7.23%                                         |

From the data that was generated, there is not one clear attribute that signifies a customer will or will not default on a loan. Rather, certain situations in each of the attributes slightly increase the chance that a customer will default on a loan. While it is shown that customer's that have children or are unmarried or in a civil partnership have the highest likelihood for defaulting on a loan, the other attributes must also be taken into account. 

For example, a customer may be in a civil partnership and have kids, but fall under the wealth income bracket and are using the loan to fund a new house. The overall risk of this customer defaulting on a loan would fall into a medium risk area as the low risk of being in the wealthy income bracket and using the loan for housing would offset the risk associated with children and being in a civil partnership.

All of these attributes must be carefully evaluated for prospective customers. Risk must not be scored on one individual attribute, but all attributes. Customer's who fall into the riskier categories for all four of the attributes evaluated herein should be considered a risky customer from the bank's perspective.